# Simple IEX Stock Fetcher

The IEX free message quota only gets one year of data for 200 stocks, so run this, fetcher stock data and save to disk every month.


## Test api

In [1]:
import iex
import key
iex.init(key.test_token, api='sandbox')
aapl = iex.Stock('AAPL')
aapl.chart('5d', chartCloseOnly=True)

,close,volume,change,changePercent,changeOverTime
date,,,,,
2019-12-02,271.69,24432805,0.00,0.0000,0.000000
2019-12-03,266.28,30088737,-4.83,-1.8450,-0.018550
2019-12-04,273.66,17175578,2.31,0.9090,-0.009207
2019-12-05,269.96,19239028,3.84,1.4762,0.005525
2019-12-06,282.17,27811895,5.13,1.9364,0.024948


In [2]:
aapl.dividends('1y')

,paymentDate,recordDate,declaredDate,amount,flag,currency,description,frequency,date
exDate,,,,,,,,,
2019-11-17,2019-11-23,2019-11-16,2019-11-01,0.79,Chas,USD,Od Seyhrnrsiaar,reqrltauy,2019-12-07
2019-08-10,2019-08-22,2019-08-21,2019-08-07,0.78,aCsh,USD,irraSyn ershOad,yaruqtrle,2019-12-07
2019-05-20,2019-05-29,2019-05-17,2019-05-13,0.81,hasC,USD,s eahndrSyrrOai,tarleyuqr,2019-12-07
2019-02-11,2019-02-17,2019-02-22,2019-02-01,0.74,ahsC,USD,raahedyOsn rriS,erqulytar,2019-12-07


In [14]:
# iex.init(key.token, api='cloud')
# aapl.dividends('1y')

,paymentDate,recordDate,declaredDate,amount,flag,currency,description,frequency,date
exDate,,,,,,,,,
2019-11-07,2019-11-14,2019-11-11,2019-10-30,0.77,Cash,USD,Ordinary Shares,quarterly,2019-12-07
2019-08-09,2019-08-15,2019-08-12,2019-07-30,0.77,Cash,USD,Ordinary Shares,quarterly,2019-12-07
2019-05-10,2019-05-16,2019-05-13,2019-04-30,0.77,Cash,USD,Ordinary Shares,quarterly,2019-12-07
2019-02-08,2019-02-14,2019-02-11,2019-01-29,0.73,Cash,USD,Ordinary Shares,quarterly,2019-12-07


In [2]:
iex.Stock('AIRT').splits('5y')

,declaredDate,ratio,toFactor,fromFactor,description,date
exDate,,,,,,
2019-06-18,2019-06-13,0.673858,3,2,i-rp2fS t3-ol,2019-12-07
2019-06-20,2019-06-08,0.672697,3,2,ptoiSr- 23-lf,2019-12-07


# Fetch Symobols

In [10]:
# 这东西得每个月更新下
iex.init(key.token, api='cloud')
symbols = iex.Reference.symbols()
symbols.to_csv("./symbols.csv")
iex.init(key.test_token, api='sandbox')

type	refers to the common issue type

    ad - ADR
    re - REIT
    ce - Closed end fund
    si - Secondary Issue
    lp - Limited Partnerships
    cs - Common Stock
    et - ETF
    wt - Warrant
    oef - Open Ended Fund
    cef - Closed Ended Fund
    ps - Preferred Stock
    ut - Unit
    struct - Structured Product

In [21]:
symbols.groupby('type').symbol.count()

type
ad         402
cef        506
cs        4536
et        2339
ps         762
rt          23
struct       2
temp         1
ut         116
wt         177
Name: symbol, dtype: int64

# Fetcher Functions

In [3]:
import os, sys
import pandas as pd
chart_range = [(5,'5d'), (20,'1m'), (75,'3m'), (165,'6m'), (341,'1y'), (715,'2y'), (99999, '5y')]

def range_since_last(df, update_date_col):
    if update_date_col is None:
        update_at = df.index[-1]
    else:
        update_at = df[update_date_col].iloc[-1]
    update_at = df[update_date_col].iloc[-1]
    update_at = pd.Timestamp(update_at)
    days = pd.Timedelta(pd.datetime.now() - update_at).days
    bdays = len(pd.bdate_range(update_at, pd.datetime.now())) - 1
    if bdays <= 0:
        return None
    for range_, param in chart_range:
        if days < range_:
            return param
    return '5y'

def fetch_on_demand(filename, index_col, update_date_col, fetch_call):
    if os.path.exists(filename):
        df = pd.read_csv(filename, index_col=index_col, parse_dates=True)
        param = range_since_last(df, update_date_col)
        sys.stdout.write(str(param) + "      ")
        if param is None:
            return
        # fetcher missing period
        try:
            df_append = fetch_call(param)
        except EOFError:  # empty data
            # update last fetch date only
            df.loc[df.index[-1], update_date_col] = pd.datetime.now()
            df.to_csv(filename)
            return
        df = df_append.combine_first(df)
        df.sort_index(inplace=True)
        df.to_csv(filename)
    else:
        sys.stdout.write('new 5y     ')
        try:
            df = fetch_call('5y')
        except EOFError:  # empty data
            # save last fetch date to csv
            pd.DataFrame([[np.nan, pd.datetime.now()]], columns=[index_col, update_date_col]).to_csv(
                filename, index=False)
            return
        df.sort_index(inplace=True)
        df.to_csv(filename)

## Fetch Adjustment

In [59]:
import numpy as np
symbols = pd.read_csv('./symbols.csv', index_col='date', parse_dates=True)
symbols = symbols[(symbols.type == 'ad') | (symbols.type == 'cs') & (symbols.exchange != 'OTC')]
symbols = symbols.symbol.values
symbols = np.append(symbols, ['SPY', 'QQQ'])
symbols, symbols.shape

(array(['A', 'AA', 'AACG', ..., 'ZYXI', 'SPY', 'QQQ'], dtype=object), (4939,))

In [7]:
from tqdm.notebook import tqdm

# iex.init(key.test_token, api='sandbox')
iex.init(key.token, api='cloud')
 
for k in tqdm(symbols):
    sys.stdout.write('\r{}   '.format(k))
    filename = "./dividends/{}.csv".format(k)
    fetch_on_demand(filename, 'exDate', 'date', iex.Stock(k).dividends)
print('ok')

QQQ   new 5y        
ok


In [6]:
from tqdm.notebook import tqdm

# iex.init(key.test_token, api='sandbox')
iex.init(key.token, api='cloud')

for k in tqdm(symbols):
    sys.stdout.write('\r{}   '.format(k))
    filename = "./splits/{}.csv".format(k)
    fetch_on_demand(filename, 'exDate', 'date', iex.Stock(k).splits)
print('ok')

QQQ   None         ne      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      
ok


iex获取的数据还是有错，比如AIRT，要yahoo交叉验证

## Fetch stocks

In [10]:
import requests
import pandas as pd
import time

In [6]:
# todo read top 600 stock from history

In [23]:
# iex.init(key.test_token, api='sandbox')
iex.init(key.token, api='cloud')

# save historical price to disk
for k in spy:
    k = correction[k] if k in correction else k
    if k =='skip..': continue
    sys.stdout.write('\r{}   '.format(k))
    filename = "./daily/{}.csv".format(k)
    fetch_on_demand
    
    if os.path.exists(filename):
        # fetcher missing period
        df = pd.read_csv(filename, index_col='date', parse_dates=True)
        days = pd.Timedelta(pd.datetime.now() - df.index[-1]).days
        bdays = len(pd.bdate_range(df.index[-1], pd.datetime.now())) - 1
        for day, param in chart_range:
            if bdays <= 0:
                break
            elif days < day:
                print(param)
                df_append = iex.Stock(k).chart(param)
                pd.concat([df, df_append], sort=True).to_csv(filename)
                break
    else:
        print('full')
        iex.Stock(k).chart('5y').to_csv(filename)

In [ ]:
etfs = pd.read_html(requests.get('https://etfdailynews.com/etf/spy/', headers={'User-agent': 'Mozilla/5.0'}).text,
             attrs={'id': 'etfs-that-own'})
spy = [x for x in etfs[0].Symbol.values.tolist() if isinstance(x, str)]